# Code ideas

In [62]:
using Random
rng = Random.MersenneTwister()
using BenchmarkTools: @btime
include("RBM_VMC/includeall.jl")

vmc (generic function with 1 method)

## Initialization

In [63]:
wf = SimpleGaussian(0.5, [1.0, 1.0, 1.0])

SimpleGaussian{StaticArrays.SVector{3, Float64}}(0.5, [1.0, 1.0, 1.0], [1.0, 1.0, 1.0])

In [64]:
num = 10
dims = 3
positions = Random.rand(rng, num*dims);

In [65]:
particles = Particles(dims, num, rng, wf);

In [66]:
idx = 1:3

1:3

In [69]:
@time qf = QF(particles, idx, wf)

  0.000005 seconds (2 allocations: 224 bytes)


3-element Vector{Float64}:
 -1.5363079124152828
  0.04862535097464704
 -1.5232877567398804

In [60]:
particles.temp_vec .= [0.0, 0.0, 0.0]

3-element StaticArrays.MVector{3, Float64} with indices SOneTo(3):
 0.0
 0.0
 0.0

In [61]:
qf

3-element StaticArrays.MVector{3, Float64} with indices SOneTo(3):
  1.3356908841545083
 -0.7022602938846747
 -1.0619148474398026

In [ ]:
struct RBM
    M::Int64
    N::Int64
    W::Matrix{Float64}
    a::Vector{Float64}
    b::Vector{Float64}
    σ::Float64
end
function RBM(visible, hidden, σ, rng::AbstractRNG)
    W = Random.rand(rng, Float64, (visible, hidden))
    a = Random.rand(rng, Float64, visible)
    b = Random.rand(rng, Float64, hidden)
    return RBM(visible, hidden, W, a, b, σ)
end

## RBM

In [ ]:
rbm = RBM(length(positions), 10, 0.5, rng);

In [ ]:
function eval(X, rbm)
    sum1 = 0.0
    for i in 1:rbm.M
        sum1 += (X[i] - rbm.a[i])^2
    end
    prod = exp(-sum1 / (2.0 * rbm.σ^2))
    for j in 1:rbm.N
        sum2 = 0.0
        for i in 1:rbm.M
            sum2 += X[i] * rbm.W[i, j]
        end
        prod *= 1.0 + exp(rbm.b[j] + sum2 / rbm.σ^2)
    end
    return prod
end

In [ ]:
@time eval(positions, rbm)

In [ ]:
function sigmoid(x)
    return exp(x) / (1.0 + exp(x))
end

In [ ]:
function kinetic(X, rbm)
    dder = 0.0
    for m in 1:rbm.M
        sum1 = 0.0
        sum2 = 0.0
        for n in 1:rbm.N
            sum3 = 0.0
            for mm in 1:rbm.M
                sum3 += X[mm] * rbm.W[mm, n]
            end
            sig_inp = rbm.b[n] + sum3 / rbm.σ^2
            
            wsig = rbm.W[m, n] * sigmoid(sig_inp)
            sum1 += wsig
            sum2 += wsig * rbm.W[m, n] * sigmoid(-sig_inp)
        end
        dlnΨ = rbm.a[m] - X[m] + sum1
        d2lnΨ = -rbm.σ^2 + sum2
        dder += dlnΨ^2 + d2lnΨ
    end
    return -0.5 * dder / rbm.σ^4
end

In [ ]:
import StaticArrays as sa
b = sa.SizedVector{2}

In [ ]:
@btime kinetic(positions, rbm)

In [ ]:
function paramDer(X, rbm)
    da = 1 / rbm.σ^2 * (X .- rbm.a)
    sig = rbm.b
    for m in 1:rbm.M
        for n in 1:rbm.N
            sig[n] += X[m] .* rbm.W[m, n] / rbm.σ^2
        end
    end
    sig .= sigmoid.(sig)
    db = sig
    dW = X * la.transpose(sig) ./ rbm.σ^2
    return da, db, dW
end

In [ ]:
@time gradient = paramDer(positions, rbm);

In [ ]:
function QF(X, rbm, idx)
    qf = zeros(length(idx))
    
    for (i, m) in enumerate(idx)
        sum1 = 0.0
        for n in 1:rbm.N
            sum2 = 0.0
            for mm in 1:rbm.M
                sum2 += X[mm] * rbm.W[mm, n]
            end
            sig_inp = rbm.b[n] + sum2 / rbm.σ^2
            sum1 += rbm.W[m, n] * sigmoid(sig_inp)
        end
        qf[i] = rbm.a[m] - X[m] + sum1
    end
    return qf
end

In [ ]:
@time QF(positions, rbm, 7:9)